## Peut-on se passer de Java ?

L'API de Hadoop MapReduce est en Java, ce qui permet d'explorer la totalité des possibilités de la plateforme. Toutefois, l'écriture du code Java est longue (plusieurs classes, des dizaines de lignes) et nécessite la compilation et la création d'un JAR.

Si on n'a pas besoin de constructions trop élaborées (ex : on veut juste faire un map suivi d'un reduce), Hadoop propose une application *wrapper* permettant de faire appel à des scripts extérieurs pour effectuer les étapes de map et reduce. Ce *wrapper* fait partie de l'API Streaming.

Ainsi, vous pouvez écrire des codes en Python, Bash, ou tout autre langage et les utiliser, du moment où ces codes acceptent des entrées **clé, valeur** et produisent des sorties **clé, valeur**.

## WordCount en Python

Nous allons donc refaire le code WordCount en utilisant deux codes Python, l'un pour le mapper, l'autre pour le reducer. 


Ce code lira les données de STDIN, les divisera en mots et produira une liste de **clé, valeur** vers STDOUT. Le script Map ne calculera pas une somme intermédiaire des occurrences d’un mot (pas de *combiner*). Au lieu de cela, il produira immédiatement des tuples {*word*, 1} même si un mot spécifique peut apparaître plusieurs fois dans l'entrée. Dans notre cas, nous laissons l'étape de réduction effectuer le décompte final.

Ce deuxième fichier lira les résultats de *mapper.py* à partir de STDIN et additionnera les occurrences de chaque mot à un décompte final, puis affichera ses résultats dans STDOUT.

Attention : le format de sortie de *mapper.py* et le format d'entrée attendu de *reducer.py* doivent correspondre.

Avant de lancer ce code sur Hadoop, on peut le tester directement sur un fichier texte pour voir si ça marche. 

In [ ]:
! chmod +x mapper.py
! chmod +x reducer.py

In [ ]:
! cat resources/datasets/livres/Charles_Darwin___On_the_Origin_of_Species_1st_Edition.txt | ./mapper.py

In [ ]:
! cat resources/datasets/livres/Charles_Darwin___On_the_Origin_of_Species_1st_Edition.txt | ./mapper.py | sort -k1,1 | ./reducer.py

Les tests montrent que les scripts fonctionnent. Il faut maintenant faire appel à Hadoop. La ligne de commande ci-dessous renseigne les différents éléments pour que Hadoop Streaming puisse trouver les scripts à exécuter ainsi que les fichiers source.
* les entrées **-file** indiquent des fichiers qui doivent être copiés dans HDFS. En effet, le code à exécuter doit être accessible à toutes les machines d'un cluster.
* les entrées **--mapper** et **--reducer** indiquent qui fait quoi
* finalement, les entrées **-input** et **-output** indiquent le chemin HDFS des répertoires d'entrée et sortie des résultats

In [ ]:
! hadoop jar hadoop-*streaming*.jar \
-file ~/mapper.py    -mapper ~/mapper.py \
-file ~/reducer.py   -reducer ~/reducer.py \
-input livres -output gutenberg-output

In [ ]:
! hdfs dfs -ls gutenberg-output

In [ ]:
! hdfs dfs -cat gutenberg-output/part-00000 | head -10

### Récupérer un fichier à partir de HDFS

On a bien utilisé l'option `-put` pour mettre notre dataset dans HDFS. Souvent, le résultat d'un traitement (avec MapReduce, par exemple) sera aussi stocké dans HDFS. Dans ce cas, on peut le récupérer et copier dans notre "disque local" avec l'option `-get` :

In [ ]:
! hdfs dfs -get gutenberg-output/part-00000

In [ ]:
! ls

Excellent, vous avez pu lancer un job mapreduce en Java et en Python. Il est maintenant le temps d'écrire votre propre code. Rendez-vous sur l'**activité 4 pour retrouver l'énoncé du projet à rendre**.